# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
CityDF = pd.read_csv("output_data/cities.csv")
CityDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.7297,-155.0900,85.28,75,4,5.99,US,1630188013
1,1,ushuaia,-54.8000,-68.3000,42.46,70,40,10.36,AR,1630187893
2,2,alofi,-19.0595,-169.9187,73.47,57,86,17.07,NU,1630188014
3,3,middlebury,44.0153,-73.1673,68.76,71,90,4.61,US,1630188014
4,4,saint paul,44.9444,-93.0933,86.29,67,75,4.61,US,1630188015


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [30]:
Location = CityDF[["Lat","Lng"]]
Humidity = CityDF["Humidity"]
figure = gmaps.figure()
HeatMap = gmaps.heatmap_layer(Location, weights = Humidity, dissipating = False, max_intensity = 300, point_radius = 5)
figure.add_layer(HeatMap)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
NewDF = CityDF.loc[
    (CityDF["Cloudiness"]== 0) & 
    (CityDF["Max Temp"] < 80) & 
    (CityDF["Max Temp"] > 70) & 
    (CityDF['Wind Speed'] < 10)].dropna()     
NewDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,nkhotakota,-12.9274,34.2961,71.58,48,0,4.12,MW,1630188022
69,69,maceio,-9.6658,-35.7353,76.44,83,0,6.91,BR,1630188011
88,88,krasnyy yar,46.5331,48.3456,75.13,40,0,8.61,RU,1630188058
135,135,areni,39.7191,45.1799,75.74,46,0,5.53,AM,1630188075
174,174,usak,38.5000,29.4167,71.29,43,0,5.64,TR,1630188089


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = NewDF[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel
19,nkhotakota,MW,-12.9274,34.2961,
69,maceio,BR,-9.6658,-35.7353,
88,krasnyy yar,RU,46.5331,48.3456,
135,areni,AM,39.7191,45.1799,
174,usak,TR,38.5000,29.4167,
197,morondava,MG,-20.2833,44.2833,
220,gat,IL,31.6100,34.7642,
240,nurota,UZ,40.5614,65.6886,
309,shingu,JP,33.7333,135.9833,
389,bolshaya martynovka,RU,47.2710,41.6679,


In [28]:

searchparameters = {"radius": 5000,"types": "lodging","key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    searchparameters["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=searchparameters) 
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel,Hotel Name
19,nkhotakota,MW,-12.9274,34.2961,,Sitima Inn
69,maceio,BR,-9.6658,-35.7353,,Poutur Pousada
88,krasnyy yar,RU,46.5331,48.3456,,Cottege near Ufa
135,areni,AM,39.7191,45.1799,,Park-Hotel Khoren's Lake
174,usak,TR,38.5000,29.4167,,NaN
197,morondava,MG,-20.2833,44.2833,,La Case Bambou
220,gat,IL,31.6100,34.7642,,OlusHome
240,nurota,UZ,40.5614,65.6886,,Guest House Ruslan Nurata & Tours
309,shingu,JP,33.7333,135.9833,,Shingu UI Hotel
389,bolshaya martynovka,RU,47.2710,41.6679,,Gostinitsa


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
hotelmarker = gmaps.marker_layer(locations,info_box_content = hotel_info)
figure.add_layer(hotelmarker)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))